# Partie 1

In [1]:
import re
import pandas as pd
def convert_to_numbers(s):
   
    words_to_numbers = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
        'zero': '0'
    }
 
    pattern = re.compile(r'\b(' + '|'.join(words_to_numbers.keys()) + r')\b')
    return re.sub(pattern, lambda x: words_to_numbers[x.group()], s)
def reg_num(num_str):
    try:
        return float(num_str)
    except ValueError:
        try:
            return float(num_str.replace(',', '.'))
        except ValueError:
            return None  # Return None for invalid numbers
class Product:
    def __init__(self, qnt, price, name):
        self.qnt = reg_num(qnt) 
        self.UnitPrice =reg_num(price) 
        self.name = name

class BillGenerator:
    def __init__(self, text):
        self.bill = []
        self.text = text
        print(self.text)
        self.sentences=self.split_text_into_sentences()
        for sentence in self.sentences:
            nums = self.findNum(sentence)
            # Find the positions of "START" and "END" in the string
            start_match = re.search(nums[0], sentence)
            end_match = re.search(nums[1], sentence)
    # Extract the substrings between "START" and "END"
            start_index = start_match.end()
            end_index = end_match.start()
            substr_between = sentence[start_index:end_index]
            names = self.findName(substr_between)
            names = ' '.join(names)
            nums = [float(num) if '.' in num else convert_to_numbers(num) for num in nums]
            
            self.bill.append(Product(nums[0], nums[1], names))
            

    def printBill(self):
        # Print header
        print("Product\tQuantity\tUnitPrice\tTotalPrice")
        # Iterate over each Product in the bill
        for product in self.bill:
            # Calculate total price
            total_price = product.qnt * product.UnitPrice
            # Print product details
            print(f"{product.name}\t{product.qnt}\t\t{product.UnitPrice}\t\t{total_price}")
    def toDF(self):
    # Create a list to hold the data for each product
        bill_data = []
    
    # Iterate over each Product in the bill
        for product in self.bill:
        # Calculate total price
            total_price = product.qnt * product.UnitPrice
        # Append product details to the list
            bill_data.append([product.name, product.qnt, product.UnitPrice, total_price])
    
    # Create a DataFrame from the bill data
        bill_df = pd.DataFrame(bill_data, columns=['Product', 'Quantity', 'UnitPrice', 'TotalPrice'])
    
    # Print the DataFrame
        return bill_df


    def split_text_into_sentences(self):
    # Define the regular expression pattern to split the text at commas
        sentence_pattern = r',(?!\d)\s*|and\s*'
    
    # Split the text into sentences using the pattern
        sentences = re.split(sentence_pattern, self.text)
    
    # Remove any empty strings or whitespace-only strings from the list
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
        return sentences
        
    def findName(self,sentence):
        product_name_pattern = r'\b(?!kilo|fresh|new|your|about|also|with|for|with|also|mil|dollar)\b[A-Za-z]{3,}\b'  # Regular expression pattern for product name
        product_names = re.findall(product_name_pattern, sentence)
        return product_names

    def findNum(self,sentence):
        number_pattern = r"\b(\b\d+(?:,\d+)*(?:\.\d+)?(?:\d+(?:,\d+)*(?:\.\d+)?)?\b|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion|trillion))\b"
    
    # Use the re.findall() function to find all numbers matching the pattern
        numbers = re.findall(number_pattern, sentence)
        if len(numbers) < 2:
            numbers[0] , numbers[1] = 1 , numbers[0]
            print("here")
        sentence = re.sub(number_pattern, '', sentence)
    # Convert the numbers to floats
       # numbers = [float(num) if '.' in num else convert_to_numbers(num) for num in numbers]
    
        return numbers
        
    #def findQnt(self):
    #def findUnitPrice(self):
        
        
        

In [2]:
bill = BillGenerator("I bought three Samsung smartphones 150 $ each, four kilos of fresh banana for 1,2 dollar a kilogram and one Hamburger with 4,5 dollar")

I bought three Samsung smartphones 150 $ each, four kilos of fresh banana for 1,2 dollar a kilogram and one Hamburger with 4,5 dollar


In [3]:
bill.printBill()

Product	Quantity	UnitPrice	TotalPrice
Samsung smartphones	3.0		150.0		450.0
banana	4.0		1.2		4.8
Hamburger	1.0		4.5		4.5


In [4]:
bill.toDF()

,Product,Quantity,UnitPrice,TotalPrice
0,Samsung smartphones,3.0,150.0,450.0
1,banana,4.0,1.2,4.8
2,Hamburger,1.0,4.5,4.5


In [5]:
bill = BillGenerator("yesterday i bought from my friend one jersey for 3 dollars and not oky thay but also one new hat for 5")
bill.printBill()

yesterday i bought from my friend one jersey for 3 dollars and not oky thay but also one new hat for 5
Product	Quantity	UnitPrice	TotalPrice
jersey	1.0		3.0		3.0
hat	1.0		5.0		5.0


# Partie 2

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [18]:
from pymongo import MongoClient
# Connect to MongoDB (assuming it's running on localhost)#
client = MongoClient('mongodb://localhost:27017/')

# Select database
db = client['NLP']

# Select collection
collection = db['TP1']
# Retrieve all documents from the collection
data = [article['article_text'] for article in collection.find()]

In [19]:
print(data[0])

['تركيا توقف التبادل التجاري مع إسرائيل بسبب "المأساة الإنسانية" في غزة', '', 'علقت تركيا جميع معاملاتها التجارية مع إسرائيل بسبب حربها على غزة، مشيرة إلى "تفاقهم المأساة الإنسانية" في القطاع.', 'وقالت وزارة التجارة التركية إن هذه الإجراءات ستبقى سارية المفعول إلى أن تسمح إسرائيل بوصول المساعدات إلى غزة، بشكل "كاف وغير متقطع".', 'وبلغ حجم المبادلات التجارية بين البلدين 7 مليارات دولار، العام الماضي.', 'واتهم وزير الخارجية الإسرائيلي يسرائيل كاتس، الرئيس التركي رجب طيب أردوغان، بأنه يتصرف مثل "الدكتاتور".', 'وقال كاتس، على موقع أكس، إن أردوغان "يغفل مصلحة الشعب التركي، ومصلحة رجال الأعمال الأتراك، ويتجاهل الاتفاقات التجارية الدولية".', 'وأضاف أنه أعطى تعليمات إلى وزارة الخارجية بأن تبحث عن بدائل للتجارة مع تركيا، بالتركيز على الانتاج المحلي، والاستيراد من دول أخرى.', 'وجاء في البيان التركي أن التعليق يشمل "جميع المنتجات".', 'شرح معمق لقصة بارزة من أخباراليوم، لمساعدتك على فهم أهم الأحداث حولك وأثرها على حياتك', 'الحلقات', 'يستحق الانتباه نهاية', 'وأضاف: "تركيا ستنفذ هذه الإجراءات الجديد

## 1. one hot encoding, bag of words, TF-IDF technics on the Data vectors collected during the lab 1.

In [20]:
# Apply one hot encoding
flattened_data = [' '.join(doc) for doc in data]
cv = CountVectorizer(binary=True)
one_hot_encoded = cv.fit_transform(flattened_data)

# Apply bag of words
cv = CountVectorizer()
bag_of_words = cv.fit_transform(flattened_data)

# Apply TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(flattened_data)

# Print the shapes of the matrices
print("One Hot Encoded shape:", one_hot_encoded)
print("Bag of Words shape:", bag_of_words)
print("TF-IDF shape:", tfidf_matrix)

One Hot Encoded shape:   (0, 5479)	1
  (0, 5994)	1
  (0, 1924)	1
  (0, 1930)	1
  (0, 10046)	1
  (0, 938)	1
  (0, 4787)	1
  (0, 3378)	1
  (0, 1654)	1
  (0, 8163)	1
  (0, 7902)	1
  (0, 7777)	1
  (0, 6158)	1
  (0, 10060)	1
  (0, 1931)	1
  (0, 6306)	1
  (0, 7786)	1
  (0, 9983)	1
  (0, 1000)	1
  (0, 5722)	1
  (0, 3232)	1
  (0, 12161)	1
  (0, 11936)	1
  (0, 1929)	1
  (0, 1970)	1
  :	:
  (67, 4314)	1
  (67, 853)	1
  (67, 10731)	1
  (67, 2898)	1
  (67, 5565)	1
  (67, 5900)	1
  (67, 211)	1
  (67, 264)	1
  (67, 179)	1
  (67, 233)	1
  (67, 3637)	1
  (67, 3804)	1
  (67, 4986)	1
  (67, 4163)	1
  (67, 242)	1
  (67, 203)	1
  (67, 8042)	1
  (67, 4748)	1
  (67, 9359)	1
  (67, 1890)	1
  (67, 5373)	1
  (67, 9627)	1
  (67, 222)	1
  (67, 8698)	1
  (67, 5864)	1
Bag of Words shape:   (0, 5479)	6
  (0, 5994)	1
  (0, 1924)	1
  (0, 1930)	1
  (0, 10046)	5
  (0, 938)	14
  (0, 4787)	3
  (0, 3378)	2
  (0, 1654)	6
  (0, 8163)	27
  (0, 7902)	17
  (0, 7777)	2
  (0, 6158)	2
  (0, 10060)	1
  (0, 1931)	3
  (0, 6306)	1
  

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


word_to_visualize = 'فلسطين'

tokenized_data = []
for article in flattened_data:
    tokenized_article = word_tokenize(article)
    tokenized_article = [word if word != 'غزة' else word_to_visualize for word in tokenized_article]
    tokenized_data.append(tokenized_article)

# Apply Skip Gram

skipgram_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, sg=1)

# Apply CBOW
cbow_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, sg=0)

# Example of accessing word vectors
print("Word Vector for '{}' (Skip Gram):\n".format(word_to_visualize), skipgram_model.wv[word_to_visualize])
print("Word Vector for '{}' (CBOW):\n".format(word_to_visualize), cbow_model.wv[word_to_visualize])

In [ ]:
from gensim.models import FastText

# Apply Glove
glove_model = FastText(sentences=tokenized_data, vector_size=100, window=5, sg=1)

# Apply FastText
fasttext_model = FastText(sentences=tokenized_data, vector_size=100, window=5, sg=0)

# Example of accessing word vectors
print("Word Vector for '{}' (GloVe):\n".format(word_to_visualize), glove_model.wv[word_to_visualize])
print("Word Vector for '{}' (FastText):\n".format(word_to_visualize), fasttext_model.wv[word_to_visualize])

In [ ]:
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Function to plot vectors using t-SNE in 3D
def plot_vectors_3d(vectors, model_name):
    tsne = TSNE(n_components=3, random_state=0)
    vectors_tsne = tsne.fit_transform(vectors)
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(vectors_tsne[:, 0], vectors_tsne[:, 1], vectors_tsne[:, 2])
    ax.set_title(f'{model_name} t-SNE Plot (3D)')
    plt.show()

# Plotting all the encoded/vectorized vectors in 3D
plot_vectors_3d(one_hot_encoded.toarray(), 'One Hot Encoded')
plot_vectors_3d(bag_of_words.toarray(), 'Bag of Words')
plot_vectors_3d(tfidf_matrix.toarray(), 'TF-IDF')
# Add similar lines for Word2Vec, GloVe, and FastText vectors

# Provide a general conclusion
print("General Conclusion:")
print("Word embedding techniques like Word2Vec, GloVe, and FastText capture semantic relationships between words, making them suitable for NLP tasks.")
print("TF-IDF is useful for representing the importance of words in a document.")
print("Bag of Words and one hot encoding are simpler techniques that do not consider word order but can be effective in certain contexts.")


In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Get all the words in the dataset
all_words = set()
for article in flattened_data:
    all_words.update(word_tokenize(article))

# Apply Skip Gram
skipgram_vectors = []
for word in all_words:
    try:
        vector = skipgram_model.wv[word]
        skipgram_vectors.append(vector)
    except KeyError:
        pass
skipgram_vectors_array = np.array(skipgram_vectors).reshape(-1, 100)

# Apply CBOW
cbow_vectors = []
for word in all_words:
    try:
        vector = cbow_model.wv[word]
        cbow_vectors.append(vector)
    except KeyError:
        pass
cbow_vectors_array = np.array(cbow_vectors).reshape(-1, 100)

# Apply Glove
glove_vectors = []
for word in all_words:
    try:
        vector = glove_model.wv[word]
        glove_vectors.append(vector)
    except KeyError:
        pass
glove_vectors_array = np.array(glove_vectors).reshape(-1, 100)

# Apply FastText
fasttext_vectors = []
for word in all_words:
    try:
        vector = fasttext_model.wv[word]
        fasttext_vectors.append(vector)
    except KeyError:
        pass
fasttext_vectors_array = np.array(fasttext_vectors).reshape(-1, 100)

# Concatenate all the vector matrices into one array
all_vectors_array = np.concatenate((skipgram_vectors_array, cbow_vectors_array, glove_vectors_array, fasttext_vectors_array))

# Apply t-SNE with 3 components for 3D visualization
tsne_3d = TSNE(n_components=3, random_state=42)
vectors_tsne_3d = tsne_3d.fit_transform(all_vectors_array)

# Plot the vectors in 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(vectors_tsne_3d[:, 0], vectors_tsne_3d[:, 1], vectors_tsne_3d[:, 2])
for i, word in enumerate(all_words):
    ax.text(vectors_tsne_3d[i, 0], vectors_tsne_3d[i, 1], vectors_tsne_3d[i, 2], word, color='black')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('t-SNE Visualization of Word Embeddings in 3D')
plt.show()